# Querying

In [1]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit.lexical_graph.storage.graph.falkordb import FalkorDBGraphStoreFactory
from graphrag_toolkit.lexical_graph.storage import GraphStoreFactory
from graphrag_toolkit.lexical_graph.storage import VectorStoreFactory
from graphrag_toolkit.lexical_graph import set_logging_config

set_logging_config('INFO')

# Register the FalkorDB backend with the factory
GraphStoreFactory.register(FalkorDBGraphStoreFactory)

# Create graph and vector stores
graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

### SemanticGuidedRetriever

See [SemanticGuidedRetriever](https://github.com/awslabs/graphrag-toolkit/blob/main/docs/lexical-graph/querying.md#semanticguidedretriever).

In [2]:
from graphrag_toolkit.lexical_graph import LexicalGraphQueryEngine
from graphrag_toolkit.lexical_graph.retrieval.retrievers import RerankingBeamGraphSearch, StatementCosineSimilaritySearch, KeywordRankingSearch
from graphrag_toolkit.lexical_graph.retrieval.post_processors import SentenceReranker
from graphrag_toolkit.lexical_graph.config import GraphRAGConfig

GraphRAGConfig._user_prompt = None  # Clear the invalid ARN value
GraphRAGConfig._user_prompt_arn = "arn:aws:bedrock:us-east-1:188967239867:prompt/TSF4PI4A6C"

# Trigger proper fetch
resolved_prompt = GraphRAGConfig.user_prompt
print("Prompt resolved as text:\n", resolved_prompt)


cosine_retriever = StatementCosineSimilaritySearch(
    vector_store=vector_store,
    graph_store=graph_store,
    top_k=50
)

keyword_retriever = KeywordRankingSearch(
    vector_store=vector_store,
    graph_store=graph_store,
    max_keywords=10
)

reranker = SentenceReranker(
    batch_size=128
)

beam_retriever = RerankingBeamGraphSearch(
    vector_store=vector_store,
    graph_store=graph_store,
    reranker=reranker,
    initial_retrievers=[cosine_retriever, keyword_retriever],
    max_depth=8,
    beam_width=100
)

query_engine = LexicalGraphQueryEngine.for_semantic_guided_search(
    graph_store,
    vector_store,
    retrievers=[
        cosine_retriever,
        keyword_retriever,
        beam_retriever
    ],

)

response = query_engine.query("What are the similarities and differences between Neptune Database and Neptune Analytics?")

print(response.response)

2025-05-27 17:59:23:INFO:g.l.config     :Fetching prompt using ARN: arn:aws:bedrock:us-east-1:188967239867:prompt/TSF4PI4A6C
2025-05-27 17:59:24:INFO:g.l.config     :Successfully retrieved prompt text (first 100 chars): <question>
{query}
</question>...
Prompt resolved as text:
 <question>
{query}
</question>


/home/evanerwee/Prod/graphrag-toolkit/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-05-27 17:59:30:INFO:g.l.r.r.rerank_beam_search:Retrieved 21 new nodes through beam search.
Neptune Database and Neptune Analytics are both part of Amazon Neptune, but they serve different purposes and have distinct features. Here are the key similarities and differences:

Similarities:
1. Both are graph database services offered by Amazon Web Services [source_3.11, source_4.1, source_4.9].
2. They are designed to work with highly connected datasets and graph data [source_3.18, source_3.19].
3. Both support low-latency queries [source_1.11, source_2.20, source_3.22].

Differences:
1. Purpose:
   - Neptune Database is a serverless graph database designed for optimal scalability and availability [source_1.3, source_1.4].
   - Neptune Analytics is an analytics database engine specifically for analyzing graph databases and datasets [source_1.1, source_1.2].

2. Use cases:
   - Neptune Database is suitable for applications like fraud alerting, Customer 360, and social networking [source